In [1]:
import sys
import os
import pandas as pd
import numpy as np
# import pandas_profiling
import json
import math
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',1000)
import sklearn
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split 
from collections import Counter
from pandas.api.types import is_dict_like
import itertools
import time
import matplotlib.pyplot as plt
sys.path.append('/Users/yantingting/PycharmProjects/modeling')
import plot_tools as pl
import metrics
import database_conncet as data_con
import summary_statistics as ss
import feature_selection as fs
from data_io_utils import *
import data_processing as dp
from jinja2 import Template

In [2]:
file_path = '/Users/yantingting/Seafile/风控/模型/14 巴西/Portocred/data/'

# 数据读取

In [3]:
bank_df = pd.read_excel(os.path.join(file_path,'Portocred DATA(1).xlsx'))
bank_df.head()
bank_df.shape

,LOAN ID,DATE,BIRTH DATE (CLIENT INFORM),MARITIAL STATUS,EDUCATION DEGREE,STREET NAME,HOUSE NUMBER,ZIP CODE,STATE NAME,CITY NAME,INCOME,OCCUPATION,GENDER,EMAIL,BANK NUMBER,BANK BRANCHE,BANK NUMBER.1,BANK NAME,SERASA SCORE,BOA VISTA (BVS) SCORE,INTERNAL SCORE,RISK RANGE,IRS STATUS,SERASA NEGATIVE FLAG,BIRTH DATE (BUREAU SOURCE),ADDRESS (BUREAU SOURCE),(you can discard),NATIONALITY,NEW CLIENT,INTEREST RATE (monthly),INSURANCE TYPE,INSURANCE COVER VALUE,NOTARYS OFFICE PROTEST FLAG,RETAIL DEBT DELAY FLAG,BANK DEBT DELAY FLAG,LAWSUIT FLAG,BOUNCED CHECK FLAG,STOLEN DOCUMENTS FLAG,ACTIVE COMPANIES ON HIS NAME,SOCIETARY PARTICIPATION STATUS,CDL NEGATIVE FLAG,LOAN AMOUNT (CONTRACT),LOAN TENOR (CONTRACT),MONTHLY PAYMENT (CONTRACT),NOTARYS OFFICE PROTEST AMOUNT,BANK DEBT DELAY AMOUNT,LOAN CONTRACT STATUS,BALANCE AT MOB 0,VINTAGE APPROVAL (YYYYMM),DISBURSEMENT FLAG,APPROVAL FLAG,FPD 1,FPD 2,FPD 3,OVER_30_MOB4_BALANCE,EVER60MOB4
0,3808899799,20191017,29/05/1978,S,4,RUA SATELITE,15,38446294,MG,ARAGUARI,1500.0,Assalariados,F,jetturboaraguari@outlook.com,341,653,181875,ITAU UNIBANCO SA,582.0,NaN,NaN,FAIXA_13,FALHA,APROVA,29/05/1978,ROD JK DE OLIVEIRA KM 106,S,1,CLIENTE NOVO,9.96,C,0.0,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,0,SITUACAO DO CNPJ EM 04/10/2019: INAPTA,APROVA,832.17,15,137.38,0,0,20,1066.72,201910,1,1,1,0,0,NaN,NaN
1,3808916175,20191017,21/09/1974,S,7,RUA MANECO PEDROSO,106,99030190,RS,PASSO FUNDO,2680.0,Assalariados,F,lujoaoaguiar@gmail.com,1,3534,00270245,BANCO DO BRASIL SA,588.0,NaN,NaN,FAIXA_13,FALHA,APROVA,21/09/1974,R MANECO PEDROSO 106 C,S,1,CLIENTE NOVO,9.96,C,0.0,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,1,SITUACAO DO CNPJ EM 01/10/2019: ATIVA,APROVA,1381.86,15,238.33,0,0,20,1765.21,201910,1,1,1,1,0,NaN,NaN
2,3808778599,20191014,22/04/1980,S,5,RUA DOS OPERARIOS,157,69912126,AC,RIO BRANCO,1324.0,Assalariados,M,silas_aj@hotmail.com,104,2278,0000201606,CEF CAIXA ECONOMICA FEDERAL,567.0,NaN,NaN,FAIXA_13,FALHA,APROVA,22/04/1980,R DOS OPERARIOS 157 COMP ANTIG,S,1,CLIENTE NOVO,9.96,C,0.0,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,0,NaN,APROVA,700.00,15,114.49,0,0,20,906.20,201910,1,1,1,0,0,NaN,NaN
3,3808862384,20191014,17/06/1991,P,6,RUA SALTINHO,49,77300000,TO,DIANOPOLIS,1603.0,FuncionÃ¡rios PÃºblicos,M,vinimachado2015@gmail.com,1,1307,00310727,BANCO DO BRASIL SA,578.0,NaN,NaN,FAIXA_13,FALHA,APROVA,17/06/1991,R SALTINHO 49 C ESQUI,S,1,CLIENTE NOVO,9.96,C,0.0,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,0,NaN,APROVA,675.73,15,119.49,0,0,20,874.43,201910,1,1,1,0,0,NaN,NaN
4,3808855137,20191014,20/11/1966,C,7,RUA DOS EUCALIPTOS,30,27120347,RJ,BARRA DO PIRAI,2800.0,Aposentados e Pensionistas,M,moisescosilva@yahoo.com.br,104,177,0000259189,CEF CAIXA ECONOMICA FEDERAL,667.0,NaN,NaN,FAIXA_11,FALHA,APROVA,20/11/1966,DOS EUCALIPTOS 30,S,1,CLIENTE NOVO,9.96,C,0.0,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,0,SITUACAO DO CNPJ EM 05/07/2019: BAIXADA,APROVA,1240.22,15,201.57,0,0,20,1605.56,201910,1,1,0,0,0,NaN,NaN


(445, 56)

In [4]:
df_temp = bank_df[['LOAN ID','DATE','FPD 1']]
df_temp['FPD 1'].fillna(0,inplace=True)

/Users/yantingting/opt/anaconda3/envs/modeling/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [ ]:
table_sql = '''
    CREATE TABLE ytt_baxi(
        loanid bigint,
        DATE varchar,
        flag int
    )
    '''
Filename = df_temp
SQL_CREATE_TABLE = table_sql
data_con.upload_df_to_pg(SQL_CREATE_TABLE)
insert = """
INSERT INTO ytt_baxi
   VALUES
 {% for var in var_list %}
{{ var }},
{% endfor %}
"""
var_list = []
for cols, rows in Filename.iterrows():    
    c = tuple(rows)
    var_list.append(c)
insert_sql = Template(insert).render(var_list=var_list)[:-2]
insert_sql = insert_sql.replace('\n','')
print(1)
data_con.upload_df_to_pg(insert_sql)

In [34]:
query0 = '''
select t1.loanid,date(substring(t1.date::varchar,1,8)) as date,t1.flag,t2.*
from ytt_baxi t1 
left join (select *
from (
select *,row_number()over(partition by loan_id order by create_time desc)  as sdk_rank
from risk_brazil_t_bank_data)t
where sdk_rank = 1) t2 
on t1.loanid::varchar = t2.loan_id
'''

In [37]:
df0 = data_con.database_gn(query0)
save_data_to_pickle(df0,file_path,'SDK数据.pkl')
df0.head(1)

Successful connection


,loanid,date,flag,id,app_id,system_root_status,contact_book,app_list,gps,simulator,device_info,imei,advertisement_id,current_version,uuid,app_size,phone_charging_status,electric_quantity,system_time,available_memory_size,total_memory_size,available_capacity_size,disk_size,vpn,net_type,proxy,uid,loan_id,transaction_id,request_id,package_name,create_time,message,dt,sdk_rank
0,3808097513,2019-09-19,0,5165,f4f97eeb5249228cc9b7b934047a5dff,false,"[{""name"":""caronaa"",""phones"":["" 34991600214""]},{""name"":""B"",""phones"":[""(11) 999929161""]},{""name"":""Documentaçao"",""phones"":[""(14) 991112437""]},{""name"":""Dona Fátima Gyn"",""phones"":[""(62) 84191673""]},{""name"":""Tomaz Gyn"",""phones"":[""(62) 91548002""]},{""name"":""Wigney (Márcia)"",""phones"":[""(64) 92533391""]},{""name"":""Silvino"",""phones"":[""(98) 987221685""]},{""name"":""Caixa Postal"",""phones"":[""*100""]},{""name"":""TIM"",""phones"":[""*144""]},{""name"":""MEU PLANO"",""phones"":[""*222""]},{""name"":""Flavio peru"",""phones"":[""+51 917 214 207""]},{""name"":""Alfa Emprestimo"",""phones"":[""+55 11 33868658""]},{""name"":""Juliano Jackson"",""phones"":[""+55 11 943184178""]},{""name"":""Márcio Emprestimo"",""phones"":[""+55 11 974515295""]},{""name"":""Magi créditos"",""phones"":[""+55 11 976609720""]},{""name"":""William Bulgarelli"",""phones"":[""+55 11 987596915""]},{""name"":""Anderson Faciroli Divulgador"",""phones"":[""+55 16 992266793""]},{""name"":""Bruno Divulgador"",""phones"":[""+55 16 992322280""]},{""name"":""Fernando Talk Radio"",""phones"":[""+55 18 988187076""]},{""name"":""Edi...","[{""appName"":""Grana Fácil"",""installTime"":""2019-09-16 08:54:51:051"",""packageName"":""com.fintopia.brazilcash"",""versionCode"":20005,""versionName"":""2.0.5""},{""appName"":""Serasa"",""installTime"":""2019-06-20 03:30:40:040"",""packageName"":""br.com.serasaexperian.consumidor"",""versionCode"":523,""versionName"":""6.2.2""},{""appName"":""Shazam"",""installTime"":""2018-08-18 08:42:55:055"",""packageName"":""com.shazam.android"",""versionCode"":904800,""versionName"":""9.48.0-190906""},{""appName"":""Connectmix Ranking Musical"",""installTime"":""2019-03-12 00:13:40:040"",""packageName"":""com.connectmix.ranking.musical"",""versionCode"":10205,""versionName"":""1.2.5""},{""appName"":""WhatsApp"",""installTime"":""2017-06-06 17:27:04:004"",""packageName"":""com.whatsapp"",""versionCode"":452954,""versionName"":""2.19.244""},{""appName"":""Previsão de Tempo"",""installTime"":""2019-09-14 15:57:58:058"",""packageName"":""br.inpe.cptec.www.previsao_do_tempo"",""versionCode"":2,""versionName"":""1.0.1""},{""appName"":""Caixa"",""installTime"":""2018-08-13 10:06:17:017"",""packageName"":""br.com...","[-18.156123333333333,-47.92740333333333]",false,"{""manufacture"":""LGE"",""google_version"":""NRD90U"",""version_release"":""7.0"",""heightPixels"":""1199"",""model"":""LG-M400"",""sdk_int"":""24"",""brand"":""lge"",""device"":""ANDROID"",""board"":""mt6755"",""widthPixels"":""720""}",356950080223718,None,2.4.10,00000000-3933-78bc-0000-00000036f3bb,238,0,82,2019-09-19 07:32:27:027,424 MB,"1,83 GB","1,03 GB","23,62 GB",false,1,false,,3808097513,Ae5fe1ca43a6dacca2c2aa52a3d09aa0ca78ce61d-1568889155069,Td54be3e7d37f5373f9486b1a0f42f23f6d44325c-1568889155070,br.com.inueva.contablet.portocred,2019-09-19 10:32:36.0,None,20191016,1.0


# 自有变量衍生

In [3]:
bank_df = pd.read_excel(os.path.join(file_path,'Portocred DATA(1).xlsx'),dtype={'BANK NUMBER':str})
bank_df['email1'] = bank_df['EMAIL'].apply(lambda x: x.split('@')[-1].lower().split('.')[0])
bank_col = bank_df.columns.tolist() 
bank_col_r = [i.lower().replace(' ','_') for i in bank_col ]
bank_df.rename(columns = dict(zip(bank_col,bank_col_r)),inplace = True)
bank_df.head(1)
bank_df.shape

,loan_id,date,birth_date_(client_inform),maritial_status,education_degree,street_name,house_number,zip_code,state_name,city_name,income,occupation,gender,email,bank_number,bank_branche,bank_number.1,bank_name,serasa_score,boa_vista_(bvs)_score,internal_score,risk_range,irs_status,serasa_negative_flag,birth_date_(bureau_source),address_(bureau_source),(you_can_discard),nationality,new_client,interest_rate_(monthly),insurance_type,insurance_cover_value,notarys_office_protest_flag,retail_debt_delay_flag,bank_debt_delay_flag,lawsuit_flag,bounced_check_flag,stolen_documents_flag,active_companies_on_his_name,societary_participation_status,cdl_negative_flag,loan_amount_(contract),loan_tenor_(contract),monthly_payment_(contract),notarys_office_protest_amount,bank_debt_delay_amount,loan_contract_status,balance_at_mob_0,vintage_approval_(yyyymm),disbursement_flag,approval_flag,fpd_1,fpd_2,fpd_3,over_30_mob4_balance,ever60mob4,email1
0,3808899799,20191017,29/05/1978,S,4,RUA SATELITE,15,38446294,MG,ARAGUARI,1500.0,Assalariados,F,jetturboaraguari@outlook.com,0341,653,181875,ITAU UNIBANCO SA,582.0,NaN,NaN,FAIXA_13,FALHA,APROVA,29/05/1978,ROD JK DE OLIVEIRA KM 106,S,1,CLIENTE NOVO,9.96,C,0.0,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,NAO_CONSTAM_OCORRENCIAS,0,SITUACAO DO CNPJ EM 04/10/2019: INAPTA,APROVA,832.17,15,137.38,0,0,20,1066.72,201910,1,1,1,0,0,NaN,NaN,outlook


(445, 57)

In [4]:
eda_result = ss.eda(bank_df,file_path)

In [5]:
bank_var = bank_df[['loan_id','date','fpd_1','fpd_2','fpd_3']].copy()
bank_var['serasa_score_y'] = bank_df['serasa_score'].apply(lambda x: 0 if np.isnan(x) else 1)
bank_var['boa_vista_(bvs)_score_y'] = bank_df['boa_vista_(bvs)_score'].apply(lambda x: 0 if np.isnan(x) else 1)
bank_var['internal_score_y'] =bank_df['internal_score'].apply(lambda x: 0 if np.isnan(x) else 1)
bank_var['age'] = bank_df.apply(lambda x: int(str(x['date'])[:4])-int(x['birth_date_(client_inform)'][-4:]),axis = 1)
bank_var['social_status'] = bank_df['societary_participation_status'].apply(lambda x: 0 if isinstance(x,float) else 1)
bank_var['active_companies'] = bank_df['active_companies_on_his_name']
bank_var['serasa_score'] = bank_df['serasa_score']
bank_var['boa_vista_score'] = bank_df['boa_vista_(bvs)_score']
bank_var['internal_score'] = bank_df['internal_score']
bank_var['income'] = bank_df['income']
dummy_list = ['irs_status',
 'bank_number',
 'occupation',
 'gender',
 'education_degree',
 'state_name',
 'maritial_status',
 'email1']
frame1 = bank_df[['loan_id']].copy()
for cols in dummy_list:
    temp_list = bank_df[cols].value_counts().index.tolist()
    temp_dummy = pd.get_dummies(bank_df[cols])[temp_list]
    temp_dummy.columns = [cols + '_' + str(col) for col in temp_dummy.columns]
    frame1 = frame1.join(temp_dummy)
bank_var = bank_var.merge(frame1,on ='loan_id')
# bank_var.fillna(-1,inplace = True)
bank_var.shape
bank_var.head()

(445, 87)

,loan_id,date,fpd_1,fpd_2,fpd_3,serasa_score_y,boa_vista_(bvs)_score_y,internal_score_y,age,social_status,active_companies,serasa_score,boa_vista_score,internal_score,income,irs_status_FALHA,irs_status_SEGUE,bank_number_0341,bank_number_0237,bank_number_0104,bank_number_0033,bank_number_0001,bank_number_0041,bank_number_0756,bank_number_0077,bank_number_0655,bank_number_0021,bank_number_0748,bank_number_0037,bank_number_0070,bank_number_0151,occupation_Assalariados,occupation_FuncionÃ¡rios PÃºblicos,occupation_Aposentados e Pensionistas,occupation_Capitalista rend aplic / aluguel,gender_M,gender_F,education_degree_5,education_degree_7,education_degree_6,education_degree_3,education_degree_8,education_degree_4,education_degree_2,state_name_SP,state_name_RJ,state_name_MG,state_name_RS,state_name_SC,state_name_BA,state_name_PR,state_name_DF,state_name_GO,state_name_CE,state_name_PA,state_name_PE,state_name_MT,state_name_PI,state_name_AM,state_name_TO,state_name_RN,state_name_MA,state_name_ES,state_name_AC,state_name_PB,state_name_AL,state_name_MS,state_name_SE,state_name_AP,state_name_RO,maritial_status_S,maritial_status_C,maritial_status_D,maritial_status_P,maritial_status_O,maritial_status_V,maritial_status_J,email1_gmail,email1_hotmail,email1_yahoo,email1_outlook,email1_bol,email1_live,email1_terra,email1_uol,email1_oi,email1_agenpol
0,3808899799,20191017,1,0,0,1,0,0,41,1,0,582.0,NaN,NaN,1500.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,3808916175,20191017,1,1,0,1,0,0,45,1,1,588.0,NaN,NaN,2680.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3808778599,20191014,1,0,0,1,0,0,39,0,0,567.0,NaN,NaN,1324.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,3808862384,20191014,1,0,0,1,0,0,28,0,0,578.0,NaN,NaN,1603.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,3808855137,20191014,0,0,0,1,0,0,53,1,0,667.0,NaN,NaN,2800.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [6]:
drop_list = ['loan_id','date','fpd_1','fpd_2','fpd_3']
var_dict_bank = dp.VarDict(bank_var.drop(drop_list, axis = 1), data_sorce='bank')
var_dict_bank.replace('uint8','int64',inplace=True)
var_dict_bank.to_csv(os.path.join(file_path, 'var_dict_bank.csv'), index = False)
var_dict_bank.shape
save_data_to_pickle(bank_var,file_path,'bank_var.pkl')

(82, 4)

# 分训练集和验证集

In [7]:
X_train, X_test, y_train, y_test = train_test_split(bank_df['loan_id'],  bank_df['fpd_2'],
                                                    test_size=0.25, random_state=2,
                                                   shuffle = True,stratify = bank_df['fpd_2'])
df_train =  pd.merge(X_train.to_frame(), y_train.to_frame(), left_index=True, right_index=True)
df_train['sample_flag'] = 0
df_train.shape
df_test =  pd.merge(X_test.to_frame(), y_test.to_frame(), left_index=True, right_index=True)
df_test['sample_flag'] = 1
flag_df = pd.concat([df_train,df_test],axis=0) 
flag_df['sample_flag'].value_counts()
flag_df.to_csv(os.path.join(file_path,'flag_df.csv'),index = False)

(333, 3)

0    333
1    112
Name: sample_flag, dtype: int64

In [9]:
pd.crosstab(flag_df['sample_flag'],flag_df['fpd_2'])

fpd_2,0,1
sample_flag,,
0,227,106
1,77,35


In [ ]:
106/